In [2]:
import sys
sys.path.append(rf'/nfs/home/genovese/thesis-wildfire-genovese/src/')
from importlib import reload
import utils
reload(utils)
from utils import *
data_folder = '/nfs/home/genovese/thesis-wildfire-genovese/data/'

In [ ]:
confini = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/clean_data/confini_piemonte/confini_piemonte.shp')

In [ ]:
meteo = dict()
    
for a in list(range(2000, 2025)):
    meteo[a] = gpd.read_file(f'/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/weather_forecast/{a}.geojson')

## Ordinary Kriging

In [ ]:
# from pykrige.ok import OrdinaryKriging
    
# gdf = meteo[2000]
    
# mask = gdf['tmedia'].dropna().index 
# day = gdf.loc[mask, :][gdf.loc[mask, :].data == '2000-01-03'].to_crs(epsg=4326)
    
# # 5. Extract lat, lon, and temperature data
# lons = day.geometry.x.values
# lats = day.geometry.y.values
# temps = day['tmedia'].values
    
# # 6. Interpolate using kriging from LandSklim
# result = OrdinaryKriging(lons, lats, temps, verbose=True, variogram_model='gaussian')
    
# grid_lons = np.linspace(confini.total_bounds[0], confini.total_bounds[2], 1000)
# grid_lats = np.linspace(confini.total_bounds[1], confini.total_bounds[3], 1000)
   
# z, ss = result.execute('grid', grid_lons, grid_lats)

In [ ]:
# fig, ax = plt.subplots()
    
# plt.pcolormesh(grid_lons, grid_lats, z)
# # confini.plot(ax=ax, color='lightgray', alpha=0.5)
# plt.scatter(lons, lats, c=temps, edgecolors='k', marker='o', label='Data points')
    
# plt.colorbar(label='Predicted Value')
# plt.title('Kriging Interpolation')
# plt.xlabel('X Coordinate')
# plt.ylabel('Y Coordinate')
# plt.legend(loc='best')
# plt.show()

In [ ]:
# from sklearn.metrics import mean_squared_error
# import itertools
    
# # Define variogram models for the grid search
# variogram_models = {'hole-effect', 'spherical', 'exponential', 'gaussian', 'linear', 'power'}
# ranges = [10, 20, 30]
# slopes = [0.1, 0.3, 0.5]
# sills = [1, 5, 7]
# nuggets = [0, 2, 5]
# scales = [1,3,5]
# exponent = [1,2,3]
    
    
# # Grid search to find the best variogram parameters
# best_model = None
# best_rmse = float('inf')
# best_params = None
    
# for var_model, slope_val, range_val, sill_val, nugget_val, scale_val, exponent_val in list(itertools.product(variogram_models, ranges, slopes, sills, nuggets
# , scales, exponent)):
#     # Create an OrdinaryKriging object
#     OK = OrdinaryKriging(lons, lats, temps, variogram_model=var_model, 
#                                          variogram_parameters={
#                                                  'slope': slope_val, 
#                                                  'range': range_val, 
#                                                  'sill': sill_val, 
#                                                  'nugget': nugget_val,
#                                                  'scale': scale_val,
#                                                  'exponent': exponent_val},
#                                          verbose=False, enable_plotting=False)
                    
#     z_pred, ss = result.execute('points', lons, lats)
    
#     rmse = np.sqrt(mean_squared_error(temps, z_pred.flatten()))
                    
#     # print(f\Model: {var_model}, Range: {range_val}, Sill: {sill_val}, Nugget: {nugget_val}, RMSE: {rmse}\)
    
#     # Update best model if current RMSE is better
#     if rmse < best_rmse:
#         best_rmse = rmse
#         best_model = var_model
#         best_params = (range_val, sill_val, nugget_val)
    
#     # Output the best model and parameters
# print('Best Model:', best_model)
# print('Best Parameters:', best_params)

## Regression Kriging

`Predictive variables:` day (GG + MM), height (Quota (m s.l.m))
<br> <br>
`Target variables:` 'tmedia', 'tmax', 'tmin', 'ptot', 'vmedia', 'vraffica', 'settore_prevalente', 'tempo_permanenza',
           'durata_calma', 'umedia', 'umin', 'umax', 'rtot', 'hdd_base18', 'hdd_base20', 'cdd_base18'

In [ ]:
def month_day_to_day_number(year, month, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    day_of_year = sum(days_per_month[:month - 1]) + day
    return day_of_year
    
sector_to_degree = {
            'Calma di vento': 0,
            'N': 1,
            'NNE': 2,
            'NE': 3,
            'ENE': 4,
            'E': 5,
            'ESE': 6,
            'SE': 7,
            'SSE': 8,
            'S': 9,
            'SSW': 10,
            'SW': 11,
            'WSW': 12,
            'W': 13,
            'WNW': 14,
            'NW': 15,
            'NNW': 16
        }

In [ ]:
def DataLoader(gdf, vars = ['tmedia', 'tmax', 'tmin', 'ptot', 'vmedia', 'vraffica', 'settore_prevalente', 'tempo_permanenza',
           'durata_calma', 'umedia', 'umin', 'umax', 'rtot', 'hdd_base18', 'hdd_base20', 'cdd_base18']):
        result = dict.fromkeys(vars)
       
        aux = gdf
        aux['settore_prevalente'] = aux.loc[:, 'settore_prevalente'].map(sector_to_degree)
        aux['day'] = aux.apply(lambda x: month_day_to_day_number(x['YYYY'], x['MM'], x['DD']), axis=1)
     
        for key in result.keys():
            foo = aux[['day', 'Quota (m s.l.m.)', key, 'geometry']]
            mask = foo[key].dropna().index 
            foo = foo.loc[mask, :]
     
            result[key] = foo.rename(columns={'Quota (m s.l.m.)': 'height'})
     
        return result

In [ ]:
meteo_per_kriging = dict()
for a, gdf in meteo.items():
    meteo_per_kriging[a] = DataLoader(gdf)

In [ ]:
import pickle

with open('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/data_for_weather_kriging.pkl', 'wb') as f:
     pickle.dump(meteo_per_kriging, f)

---

#### Ignore

In [ ]:
# from shapely.geometry import Point

# gdf = altimetria.to_crs(epsg=4623)

# minx, miny, maxx, maxy = gdf.total_bounds

# # 2. Generate a grid of points
# # Set the step size (spacing between grid points)
# step = 0.01  # Change this to the desired resolution

# # Create the x and y coordinates of the grid
# x_coords = np.arange(minx, maxx, step)
# y_coords = np.arange(miny, maxy, step)

# # Generate a meshgrid of points (all combinations of x and y)
# grid_points = np.array(np.meshgrid(x_coords, y_coords)).T.reshape(-1, 2)

# # 3. Create Point objects and check if they are inside the polygons
# points = [Point(x, y) for x, y in grid_points]

# points_within_polygons = [point for point in tqdm(points) if gdf.contains(point).any()]

# def get_polygon_value(gdf, point):
#     for idx, polygon in gdf.iterrows():
#         if polygon['geometry'].contains(point):
#             return polygon['MEDIANA']
#     return None

# # Apply the function to all points
# values = [get_polygon_value(gdf, point) for point in tqdm(points_within_polygons)]

# # 5. Create a GeoDataFrame with points and associated polygon values
# points_gdf = gpd.GeoDataFrame(
#     geometry=points_within_polygons,
#     data={'polygon_value': values},
#     crs=gdf.crs
# )

# points_gdf.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/altimetric_grid.geojson')

# # Show the first few points
# print(points_gdf.head())


---

### Set points for aimed kriging

In [ ]:
target = separate_date(gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/nicola/piedmont_2012_2024_fa.geojson'), 'initialdate')

In [ ]:
def month_day_to_day_number(year, month, day):
    feb = 28
    if year in list(range(2000, 2025, 4)):
        feb += 1
            
    days_per_month = [31, feb, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    day_of_year = sum(days_per_month[:month - 1]) + day
    return day_of_year

In [ ]:
target['day_of_year'] = target.apply(lambda x: month_day_to_day_number(x['YYYY'], x['MM'], x['DD']), axis=1)

In [ ]:
altimetria = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/gathering_geojson/altimetria_per_circoscrizione.geojson').to_crs(target.crs)

In [ ]:
from shapely.geometry import Point

target['ignition_point'] = [Point(x, y) for x, y in tqdm(zip(target.point_x, target.point_y))]

def get_polygon_value(gdf, point, value):
    for idx, polygon in gdf.iterrows():
        if polygon['geometry'].contains(point):
            return polygon[value]
    return None

# Apply the function to all points
values = [get_polygon_value(altimetria, point, 'MEDIANA') for point in tqdm(target['ignition_point'])]

# 5. Create a GeoDataFrame with points and associated polygon values
points_gdf = gpd.GeoDataFrame(
    geometry='ignition_point',
    data={'height': values},
    crs=altimetria.crs
)

for_kriging = gpd.GeoDataFrame(target[['day_of_year', 'ignition_point']], geometry='ignition_point'
                               ).merge(points_gdf, on='geometry', how='inner')

for_kriging.to_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_weatehr_grid.geojson')

---

In [ ]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/data_for_weather_kriging.pkl', 'rb') as f:
     loaded_dict = pickle.load(f)
     
print(list(loaded_dict.keys()))

ignitions = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_weatehr_grid.geojson'
                       ).rename(columns={'day_of_year': 'day'}) 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from pykrige.rk import RegressionKriging

def rk_train_pred(X_train, X_pred, target_col):
    RK = RegressionKriging(regression_model = GradientBoostingRegressor(), method='universal', variogram_model = 'spherical', verbose=1)
       
    RK.fit(p = X_train[['height']], x = np.transpose(np.array([X_train.geometry.x, X_train.geometry.y])), y = X_train[target_col])
       
    y_pred = RK.predict(p = X_pred[['height']], x = np.transpose(np.array([X_pred.geometry.x, X_pred.geometry.y])))

    return y_pred



def yearly_forecast(loaded_dict, target, year):
    
    centraline = loaded_dict[year]
    tt = target[target['YYYY'] == year]

    forecasts = tt.copy()
    
    for var, gdf in tqdm(centraline.items(), desc=f'Running year: {year}'):

        parameter_forecast = gpd.GeoDataFrame()

        for day, X_pred in tqdm(tt.groupby('day'), desc=f'Running variable: {var}'):
            aux = X_pred.copy()
            X_train = gdf[gdf.day == day]
            y_pred = rk_train_pred(X_train, X_pred, var)
            aux = pd.concat([aux, y_pred], axis=1)
            parameter_forecast = pd.concat([parameter_forecast, aux], axis=0, ignore_index=True)

        forecasts.merge(parameter_forecast, on='geometry', how='inner')

    return forecasts



weather_forecast = dict()

for year in tqdm(ignitions.YYYY.unique()):
    weather_forecast[year] = yearly_forecast(loaded_dict, ignitions, year)

---

In [1]:
import sys 
sys.path.append(rf"/nfs/home/genovese/thesis-wildfire-genovese/src")
from importlib import reload
import utils
reload(utils)
from utils import *
import weather_forecast_utils
reload(weather_forecast_utils)
from weather_forecast_utils import *
data_folder = "/nfs/home/genovese/thesis-wildfire-genovese/data/"

In [2]:
import pickle 
with open('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/data_for_weather_kriging.pkl', 'rb') as f:
     loaded_dict = pickle.load(f)

ignitions = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/data/data_loader_for_kriging/kriging_weather_grid.geojson'
                       ).rename(columns={'day_of_year': 'day'}) 

# save_clean_data(yearly_forecast(loaded_dict, ignitions, 2016), str(2016), '/nfs/home/genovese/thesis-wildfire-genovese/data/output_weather_kriging')

In [ ]:
yearly_forecast(loaded_dict, ignitions, 2016)

---

In [4]:
gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/meteo_2023.geojson')

,day,YYYY,height,tmedia,tmax,tmin,ptot,vmedia,vraffica,settore_prevalente,tempo_permanenza,durata_calma,umedia,umin,umax,rtot,hdd_base18,hdd_base20,cdd_base18,geometry
0,101,2023,213.0,13.230705,20.158412,6.883750,1.378973e-03,1.622872,6.955506,9.687054,414.539682,248.834683,68.521815,46.106668,90.240610,15.246044,4.686947,6.729455,0.0,POINT (911536.68982 5703104.65851)
1,109,2023,395.0,14.775805,22.877665,7.122663,-8.449210e-03,2.482651,9.722485,7.833310,416.941652,10.547777,49.178477,23.107929,86.554401,24.966531,2.807913,5.353116,0.0,POINT (892993.82569 5561082.1195)
2,99,2023,1375.0,4.291810,8.578450,0.012318,-9.195517e-06,2.779627,7.148843,8.885296,430.898960,31.258684,70.122736,53.771789,91.903224,17.882287,13.477080,15.477080,0.0,POINT (903686.34808 5760387.24026)
3,86,2023,394.0,11.869276,17.542753,5.364078,-1.032420e-02,5.149917,16.038527,13.835113,579.132272,6.730087,32.196838,14.223915,61.057512,21.689590,6.910775,8.910775,0.0,POINT (915429.44215 5558742.89123)
4,100,2023,340.0,11.791326,16.745255,7.001091,-5.880132e-05,1.291907,5.241868,2.654616,310.994661,55.813700,55.574239,38.378083,76.777708,18.128600,6.173773,8.173773,0.0,POINT (916183.01372 5568135.97042)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,110,2023,355.0,12.348990,16.098583,10.020745,1.620556e+00,1.964421,9.272180,5.871848,417.791182,48.865560,80.698078,58.250688,97.381560,10.588375,4.808707,7.032297,0.0,POINT (933647.58894 5555868.82402)
191,102,2023,821.0,9.033914,12.263148,6.269556,1.041202e+00,1.953632,10.034395,4.695832,397.163144,57.997637,85.015717,57.571930,98.240698,5.225679,8.583477,10.583477,0.0,POINT (811160.35764 5624752.01842)
192,107,2023,519.0,12.839117,18.153964,8.682968,5.042292e-03,2.820755,10.124378,6.510465,410.786754,18.760468,46.572588,25.873415,65.751923,19.643540,5.005724,7.057999,0.0,POINT (906164.3192 5561070.73072)
193,81,2023,370.0,13.058255,19.293825,6.879533,1.773220e-10,1.802349,5.429974,8.709642,503.857901,44.442693,63.454603,40.719715,85.015637,17.986141,3.958855,6.549303,0.0,POINT (936284.34213 5742375.43005)
